In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load a pre-trained model

## Load from HuggingFace

In [2]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

tokenizer.save_pretrained("tinyllama_f16")
model.save_pretrained("tinyllama_f16")

## Load from local repo

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    "./tinyllama_f16"
)
base = AutoModelForCausalLM.from_pretrained(
    "./tinyllama_f16",
    torch_dtype=torch.float16
).to(device)

In [4]:
prompt = "Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

base.eval()

with torch.no_grad():
    output = base.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château. Il y avait quelques maisons de pierre qui semblaient proches du château, mais elles étaient loin d’être en contact direct avec la structure principale.

Ces endroits étaient un peu étranges. La nuit était profonde et la neige les isolait de la vie quotidienne, mais ils n’avaient pas l’impression de vivre de leur propre chair.

Le père, K., savait que la neige était un excellent matériau pour les tapis. Il avait besoin de tapis de neige pour le restaurant qui faisait partie du complex


## Prepare the dataset

In [5]:
with open("chateau.txt", "r", encoding="utf-8") as f:
    txt = f.read()

# remove page numbers
txt = re.sub(r"–\s*\d+\s*–\n", "", txt)

# fix split words
txt = re.sub(r"-\n", "", txt)

# remove line breaks
txt = re.sub(r'\n(?!–)', ' ', txt)

with open("clean.txt", "w", encoding="utf-8") as f:
    f.write(txt)